In [1]:
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Poisson Surface Reconstruction

In [2]:
import os
import open3d as o3d

def set_mesh_color(mesh, color):
    if not mesh.has_vertices():
        print("Mesh has no vertices to color.")
        return
    colors = [color for _ in range(len(mesh.vertices))]
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

def process_point_cloud(ply_file, output_mesh_file, color=(0.1, 0.1, 0.7)):
    # Load point cloud
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Noise removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Surface reconstruction using Poisson method
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    # Check if the mesh has vertices
    if not mesh.has_vertices():
        print(f"No vertices found in the generated mesh for {ply_file}. Skipping.")
        return
    
    mesh.compute_vertex_normals()
    
    # Set the mesh color
    set_mesh_color(mesh, color)
    
    # Save the mesh
    o3d.io.write_triangle_mesh(output_mesh_file, mesh, write_vertex_normals=True, write_vertex_colors=True)
    
    print(f"Processed and saved: {output_mesh_file}")

def visualize_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    
    if not mesh.has_vertex_colors():
        print(f"Mesh {mesh_file} has no vertex colors. Visualizing without colors.")
    
    # Ensure the vertex colors are used in visualization
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

# Function to compute properties of the mesh
def compute_properties(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()

    # Surface area
    surface_area = mesh.get_surface_area()

    return surface_area

# Directory paths
input_ply_dir = r"D:\ply_final\one"
output_mesh_dir = r"D:\poisson"

# Ensure output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_mesh.obj"))
        
        process_point_cloud(ply_file_path, output_mesh_file, color=(0.4, 0.4, 0.4))  # Change color as needed
        #visualize_mesh(output_mesh_file)
        
        surface_area = compute_properties(output_mesh_file)
        print(f"Mesh {output_mesh_file}: Surface Area = {surface_area}")


[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_1_mesh.obj
Mesh D:\poisson\point_cloud_1_mesh.obj: Surface Area = 516684.3033567388
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_10_mesh.obj
Mesh D:\poisson\point_cloud_10_mesh.obj: Surface Area = 525370.4126904337
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_100_mesh.obj
Mesh D:\poisson\point_cloud_100_mesh.obj: Surface Area = 528409.0270672933
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_101_mesh.obj
Mesh D:\poisson\point_cloud_101_mesh.obj: Surface Area = 553215.7044541324
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\poisson\point_cloud_102_mesh.obj
Mesh D:\poisson\point_cloud_102_mesh.obj: Surface Area = 500844.89462447824
[Open3D WARNING] Write OBJ can not inclu

## Heatmap with Poisson Surface Reconstruction

In [1]:
import os
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

def set_mesh_color(mesh, depths):
    # Normalize depths between 0 and 1
    min_depth = np.min(depths)
    max_depth = np.max(depths)
    #print(f"Min depth: {min_depth}, Max depth: {max_depth}")
    normalized_depths = (depths - min_depth) / (max_depth - min_depth)
    
    # Use Matplotlib colormap
    colormap = plt.get_cmap('jet')
    colors = colormap(normalized_depths)[:, :3]  # Get RGB values (discard alpha channel)
    
    #print(f"Sample normalized depth: {normalized_depths[:10]}")
    #print(f"Sample colors: {colors[:10]}")
    
    # Ensure the colors are correctly assigned
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    #print(f"Assigned vertex colors (sample): {np.asarray(mesh.vertex_colors)[:10]}")

def process_point_cloud(ply_file, output_mesh_file):
    # Load point cloud
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Noise removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Surface reconstruction using Poisson method
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    # Check if the mesh has vertices
    if not mesh.has_vertices():
        print(f"No vertices found in the generated mesh for {ply_file}. Skipping.")
        return
    
    mesh.compute_vertex_normals()
    
    # Get depth values (distance from the camera)
    depths = np.asarray(pcd.points)[:, 2]  # Assuming z-coordinate is depth
    
    # Set the mesh color based on depth
    set_mesh_color(mesh, depths)
    
    # Save the mesh
    o3d.io.write_triangle_mesh(output_mesh_file, mesh, write_vertex_normals=True, write_vertex_colors=True)
    
    print(f"Processed and saved: {output_mesh_file}")

    # Visualize the mesh directly
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

def visualize_point_cloud(pcd):
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)

# Directory paths
input_ply_dir = r"D:\ply3"
output_mesh_dir = r"D:\depth"

# Ensure output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_mesh.obj"))
        
        # Load point cloud for color visualization
        pcd = o3d.io.read_point_cloud(ply_file_path)
        depths = np.asarray(pcd.points)[:, 2]
        colormap = plt.get_cmap('jet')
        normalized_depths = (depths - np.min(depths)) / (np.max(depths) - np.min(depths))
        colors = colormap(normalized_depths)[:, :3]
        pcd.colors = o3d.utility.Vector3dVector(colors)
        
        # Visualize point cloud colors
        visualize_point_cloud(pcd)
        
        process_point_cloud(ply_file_path, output_mesh_file)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Write OBJ can not include triangle normals.


KeyboardInterrupt: 

## Ball Pivot Algorithm Result

In [1]:
import os
import open3d as o3d
import numpy as np
import pymeshlab

def process_point_cloud(ply_file, output_mesh_file):
    # Load point cloud from PLY file
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample the point cloud to reduce the number of points
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Remove noise from the point cloud using statistical outlier removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals for the downsampled and denoised point cloud
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Surface reconstruction using the Ball-Pivoting Algorithm (BPA)
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector([radius, radius * 2, radius * 3]))
    bpa_mesh.compute_vertex_normals()
    
    # Save the BPA mesh to an OBJ file
    o3d.io.write_triangle_mesh(output_mesh_file, bpa_mesh)
    
    print(f"Processed and saved: {output_mesh_file}")

# Function to visualize a mesh
def visualize_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])

# Function to compute properties of the mesh
def compute_properties(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()

    # Surface area
    surface_area = mesh.get_surface_area()
    
    return surface_area

# Directory paths
input_ply_dir = r"D:\ply3"
output_mesh_dir = r"D:\mamaji"

# Ensure the output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud file in the input directory
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_mesh.obj"))
        
        process_point_cloud(ply_file_path, output_mesh_file)
        
        # Visualize the mesh
        visualize_mesh(output_mesh_file)
        
        # Compute properties and handle non-watertight meshes
        surface_area = compute_properties(output_mesh_file)
        print(f"Mesh {output_mesh_file}: Surface Area = {surface_area}")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\mamaji\point_cloud_1_mesh.obj
Mesh D:\mamaji\point_cloud_1_mesh.obj: Surface Area = 305370.4631916391
[Open3D WARNING] Write OBJ can not include triangle normals.
Processed and saved: D:\mamaji\point_cloud_10_mesh.obj
Mesh D:\mamaji\point_cloud_10_mesh.obj: Surface Area = 307561.8885631483


KeyboardInterrupt: 

In [ ]:
import os
import open3d as o3d
import numpy as np
import pymeshlab
import matplotlib.pyplot as plt

def process_point_cloud(ply_file, output_mesh_file):
    # Load point cloud from PLY file
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample the point cloud to reduce the number of points
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Remove noise from the point cloud using statistical outlier removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals for the downsampled and denoised point cloud
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Surface reconstruction using the Ball-Pivoting Algorithm (BPA)
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 3 * avg_dist
    bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector([radius, radius * 2, radius * 3]))
    bpa_mesh.compute_vertex_normals()
    
    # Save the BPA mesh to an OBJ file
    o3d.io.write_triangle_mesh(output_mesh_file, bpa_mesh)
    
    print(f"Processed and saved: {output_mesh_file}")

# Function to visualize a mesh
def visualize_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])

# Function to compute properties of the mesh
def compute_properties(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()

    # Surface area
    surface_area = mesh.get_surface_area()
    
    return surface_area

# Directory paths
input_ply_dir = r"D:\ply2"
output_mesh_dir = r"D:\mesh-2"

# Ensure the output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud file in the input directory
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_mesh.obj"))
        
        process_point_cloud(ply_file_path, output_mesh_file)
        
        # Visualize the mesh
        visualize_mesh(output_mesh_file)
        
        # Compute properties and handle non-watertight meshes
        surface_area = compute_properties(output_mesh_file)
        print(f"Mesh {output_mesh_file}: Surface Area = {surface_area}")


## Delaunay Triangulation Method

In [ ]:
import open3d as o3d
import numpy as np
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt

def process_point_cloud_delaunay(ply_file, output_mesh_file):
    # Load point cloud from PLY file
    pcd = o3d.io.read_point_cloud(ply_file)
    
    # Downsample the point cloud to reduce the number of points
    pcd = pcd.voxel_down_sample(voxel_size=0.02)
    
    # Remove noise from the point cloud using statistical outlier removal
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    
    # Estimate normals for the downsampled and denoised point cloud
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Convert point cloud to numpy array
    points = np.asarray(pcd.points)
    
    # Project points to 2D plane (e.g., XY plane)
    points_2d = points[:, :2]
    
    # Perform Delaunay triangulation
    delaunay = Delaunay(points_2d)
    
    # Create triangles from the Delaunay simplices
    triangles = delaunay.simplices
    
    # Create Open3D triangle mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(points)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    
    # Compute vertex normals
    mesh.compute_vertex_normals()
    
    # Save the mesh to an OBJ file
    o3d.io.write_triangle_mesh(output_mesh_file, mesh)
    
    print(f"Processed and saved: {output_mesh_file}")

# Function to visualize a mesh
def visualize_mesh(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh])

# Function to compute properties of the mesh
def compute_properties(mesh_file):
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh.compute_vertex_normals()

    # Surface area
    surface_area = mesh.get_surface_area()

    return surface_area

# Directory paths
input_ply_dir = r"D:\ply3"
output_mesh_dir = r"D:\delaunay-mesh"

# Ensure the output directory exists
os.makedirs(output_mesh_dir, exist_ok=True)

# Process each point cloud file in the input directory
for ply_file in os.listdir(input_ply_dir):
    if ply_file.endswith(".ply"):
        ply_file_path = os.path.join(input_ply_dir, ply_file)
        output_mesh_file = os.path.join(output_mesh_dir, ply_file.replace(".ply", "_delaunay_mesh.obj"))
        
        process_point_cloud_delaunay(ply_file_path, output_mesh_file)
        
        # Visualize the mesh
        visualize_mesh(output_mesh_file)
        
        # Compute properties
        surface_area = compute_properties(output_mesh_file)
        print(f"Mesh {output_mesh_file}: Surface Area = {surface_area}")
